# Notebook voor het omzetten van de met Wageningenmodellen en overstortschatter berekende afvoeren naar lateralen D_HYDRO.

## Preparation

### Import the required modules

In [ ]:
import os
from os.path import splitext, join, abspath, basename
import glob
import pandas as pd
import geopandas as gpd
import re
import matplotlib.pyplot as plt
import xml.etree.ElementTree as et 
from IPython.core.interactiveshell import InteractiveShell
from lxml import etree
import pandas as pd
InteractiveShell.ast_node_interactivity = "all"

### Set relative paths to input data

In [ ]:
# Absolute path of the script, which will serve as a base to obtain relative paths to all other used files.
path_script = os.path.abspath('')
path_data = join(os.path.dirname(path_script), r'00_Data')

# Shapefiles
laterals_shp_path = join(path_data, r'input_afvoeren\shapefiles\laterals.shp')
wagmod_shp_path = join(path_data, r'input_afvoeren\shapefiles\NAvakken.shp')

overstorten_shp_path = join(path_data, r'input_afvoeren\\shapefiles\lozingspunten.shp')
modelarea_shp_path = join(path_data, r'input_afvoeren\shapefiles\HoogeRaam.shp')

# Timeseries
laterals_data_path = join(path_data, r'input_afvoeren\timeseries\landelijk')

overstorten_data_path = join(path_data, r'input_afvoeren\timeseries\overstorten')

### Configure settings

In [ ]:
INTEGRITY = True # If true, checks if indexes of dataframes are unique.
SAVE_INTERMEDIATE_RESULTS = True # If true, saves outputs of scenarios to .csv files to check validity.
SCENARIO = 'hist' # pick one scenario ('hist', 'ghg120', etc.)



### Read the shapefiles 
- The locations of the laterals to which the must finally be coupled
- The location of the overstorten
- The polygons that are used by the Wageningen Model, to know which discharge to couple with which laterals

In [ ]:
gdf_modelarea = gpd.read_file(modelarea_shp_path)

gdf_laterals = (
    gpd.read_file(laterals_shp_path)
    .set_index('CODE', verify_integrity = True)
    [['Opp_hactar','X','Y','geometry']]
)
gdf_laterals['type'] = 'laterals'
gdf_laterals = gdf_laterals[gdf_laterals.within(gdf_modelarea.at[0,'geometry'])]

gdf_wagmod = (
    gpd.read_file(wagmod_shp_path)
    [['GAFIDEN','geometry']]
)



gdf_overstorten = gpd.read_file(overstorten_shp_path).set_index('RIODAT_ZRO', 
    verify_integrity=True 
    )[['geometry']]
gdf_overstorten['type'] = 'overstort'
gdf_overstorten = gdf_overstorten[gdf_overstorten.within(gdf_modelarea.at[0,'geometry'])]

gdf_laterals.head()
gdf_wagmod.head()
gdf_overstorten.head()


### Combine the tables of the laterals and the overstorten and save this shapefile. 
One output of the script is to create a shapefile with all the locations to which model data must be coupled, which is hereby done.

In [ ]:
# Create the shapefile containing all locations (laterals and overstorten). This is a component of the final output.
pd.concat([gdf_laterals,gdf_overstorten], 
    verify_integrity=False 
    )[['geometry','type']].to_file(join(path_data, r'output_afvoeren\shapefiles\lateralen_locations'))
    

The Wageningen Model Polygons contain values for GAFIDEN, which is an identifier used to obtain Specifieke Afvoer. Most laterals intersect a polygon, but for the 108 that do not, the join is made based on nearest polygon; a column is added containing the distance from the lateral to the nearest polygon. A new dataframe (gdf_unexactmatches) is created that contains laterals that are more than zero away from a polygon, just to easily see which ones aren't matched directly.

Unfortunately, the current delft3dfmpy environment (2.0.3) uses a fairly outdated version of geopandas. Therefore, rather than the geodataframe.sjoin_nearest() function, I am forced to use a for loop with a runtime of 30-40 seconds. This can be changed when the environment updates geopandas to a more recent version.

In [ ]:
polygons=gpd.GeoDataFrame(gdf_wagmod.geometry)
gdf_laterals_joined = gdf_laterals

for point in gdf_laterals_joined.geometry.index:
    nearest_polygon = polygons.distance(gdf_laterals_joined.geometry[point]).sort_values().index[0]
    gdf_laterals_joined.loc[point, 'GAFIDEN'] = gdf_wagmod.loc[nearest_polygon,'GAFIDEN']

    distance = polygons.distance(gdf_laterals_joined.geometry[point]).min()
    gdf_laterals_joined.loc[point, 'afstand'] = distance
    
gdf_laterals_joined.head()
gdf_unexactmatches = gdf_laterals_joined[gdf_laterals_joined['afstand']>0]

### Plot the different shapes to create an idea of what we are dealing with. 

I plotted the laterals to give an idea of what we're working with.

In [ ]:
fig, (ax1, ax2) = plt.subplots(2,1,figsize=(10, 10))
ax1.set_xlim(170000, 200000)
ax1.set_ylim(390000, 420000)
ax2.set_xlim(185000, 195000)
ax2.set_ylim(410000, 420000)

for plot in (ax1, ax2):
    
    fig=gdf_wagmod.plot(ax=plot, fc="lightyellow", edgecolor='black', lw=0.7)
    fig = gdf_modelarea.plot(ax=plot, fc='none', edgecolor='black', lw=2)
    fig=gdf_overstorten.plot(ax=plot, color = 'purple', marker='x',markersize = 5, label='Overstorten')
    fig=gdf_laterals_joined[gdf_laterals_joined['afstand']==0].plot(ax=plot, color='green', label = 'Directe match', markersize = 5)
    fig=gdf_laterals_joined[gdf_laterals_joined['afstand']>0].plot(ax=plot, color='red', label = 'Match door nabijheid', markersize = 5)
    plot.legend()
fig


## Extract, Transform and Load data

### Landelijke afvoer

In [ ]:
inputfolder = join(path_data,r'input_afvoeren/timeseries/landelijk/')

wagmodFiles = os.listdir(join(inputfolder, SCENARIO))

## Read Wageningen model output (discharges) per region, and multiply by the area belonging to each lateral
ls_wagmodoutput = []

for region in wagmodFiles:
    
    metadata_region = pd.read_csv(join(inputfolder, SCENARIO, region), nrows=1, skiprows=[0,2], header=None)[0][0]
    model = re.search('model:(.*) (?:van|met winterbui)', metadata_region).group(1)
            
    df_region = pd.read_csv(
            join(inputfolder, SCENARIO, region), 
            header=4, 
            delim_whitespace=True, 
            usecols=['-I-','-QC-'],
            )

    df_region=df_region.set_index(
            pd.to_datetime(df_region['-I-'],
            origin=pd.Timestamp('2010-11-09 00:00:00'),
            unit='h')
        ).drop(labels=['-I-'], axis='columns').rename(columns={'-QC-':model})
    
    ls_wagmodoutput.append(df_region)

# The result is a list containing dataframes as items
ls_wagmodoutput, len(ls_wagmodoutput)

# Combine the dataframes into one big dataframe
df_wagmodoutput = pd.concat(ls_wagmodoutput, axis=1, verify_integrity=True).T * 10 / 3600

# Set the index to be only the 'GAFIDEN'
for i in range(len(df_wagmodoutput.index)):
    ID = df_wagmodoutput.index[i].split(' ')[0]
    df_wagmodoutput.loc[df_wagmodoutput.index[i],'GAFIDEN'] = ID

### Multiplication
gdf_laterals_joined['Latid'] = gdf_laterals_joined.index
output_laterals = gdf_laterals_joined.merge(df_wagmodoutput, on='GAFIDEN', how='left')
output_laterals.set_index('Latid', inplace=True)
output_landelijkeafvoer = output_laterals.iloc[:,8:].multiply(output_laterals['Opp_hactar'],axis='index').T
output_landelijkeafvoer.index = pd.to_datetime(output_landelijkeafvoer.index)

if SAVE_INTERMEDIATE_RESULTS == True:
    output_landelijkeafvoer.to_csv(join(path_data, r'output_afvoeren\timeseries\landelijk', SCENARIO + '.csv'), sep = ';')

### Overstorten

#### Historic data

##### Define functions
Reading XML files can be a hassle and it doesn't improve the readability of the code. Therefore I wrapped it into a function that takes the path of an xml file and produces a list that contains a dataframe for each.

In [ ]:
def parse_xml(path_xml):
    """XML files consist of a root node that can have numerous generation of childnodes. This function takes the path to an xml from a certain year and 
    returns a list of first generation nodes, which I called series (same as in the xml files). Each Serie belongs to a specific overstort, and contain the second generation of childnodes: 
    - one Header (metadata) 
    - many Events (the actual data).
    The function returns a list of which each item is a serie, which in turn contains data and metadata for one overstort each.
    """
    # Read the xml file and extract the tree and root. Then filter to get the series branch.
    tree = etree.parse(path_xml)
    root = tree.getroot()
    list_of_series = [child for child in root.getchildren() if 'series' in child.tag]

    return list_of_series

def series_to_dataframe(serie):
    """Takes an individual Serie output by the function "parse_xml", transforms the data and metadata, 
    creates a clear dataframe.
    """
    # Extract the ID from the metadata. Used as column header in output.
    header = serie.find('{http://www.wldelft.nl/fews/PI}header')
    id = header.find('{http://www.wldelft.nl/fews/PI}stationName').text
    
    # Extract the actual data and reformat into a dataframe wtih a datetime index. 
    events = serie.findall('{http://www.wldelft.nl/fews/PI}event')
    data = pd.DataFrame([i.values() for i in events])
    index = pd.DatetimeIndex(pd.to_datetime(data[0]+' '+data[1], format="%Y-%m-%d %H:%M:%S"))
    data_indexed = pd.DataFrame({id: data[2].values}, index=index)
    data_divided = pd.to_numeric(data_indexed[id]).div(3600)
    
    return data_divided

##### Loop over all xml files and generate output

In [ ]:
if SCENARIO == 'hist':
    path_input_overstorten_hist = join(path_data,r'input_afvoeren\timeseries\overstorten\hist')
    input_files = glob.glob(path_input_overstorten_hist + '/*.xml')
    data_hist_ls = []

    # I suggest to view one of the input xml files with this code, to understand the tree and its branches.
    for path_xml in input_files:
        
        ls_data_year = []
        list_of_series = parse_xml(path_xml)

        for serie in list_of_series:
            dataframe = series_to_dataframe(serie)
            ls_data_year.append(dataframe)

        # Combine outputs series vertically, merging on the datetime index. Then add the output of this year to the list of outputs from all years.
        df_data_year = pd.concat(ls_data_year,axis='columns',verify_integrity=True)
        data_hist_ls.append(df_data_year)

    # Combine outputs from different years horizontally
    output_overstorten_hist = pd.concat(data_hist_ls,axis='index',verify_integrity=True)
    output_overstorten = output_overstorten_hist.fillna(0)

    if SAVE_INTERMEDIATE_RESULTS == True:
        output_overstorten.to_csv(join(path_data, r'output_afvoeren\timeseries\overstorten', 'hist.csv'), sep = ';')
        
elif 'ghg' in SCENARIO:
    path_input_overstorten_blokbuien = join(path_data,r'input_afvoeren\timeseries\overstorten\blokbuien')
    input_files = glob.glob(path_input_overstorten_blokbuien + '/*.xlsx')

    for file in input_files:
        excel_table = pd.read_excel(file, skiprows=3)
        index_col = excel_table['code OS']
        data = excel_table.filter(regex='Unnamed', axis='columns').div(3600) # The columns containing the data are prefixed by 'Unnamed' due to Excel structure. 
        data.columns = range(1,76)
        data.index = index_col
        data_transposed = data.T

        output_overstorten = data_transposed
        
        if SAVE_INTERMEDIATE_RESULTS == True:
            output_overstorten.to_csv(join(path_data, r'output_afvoeren\timeseries\overstorten', basename(splitext(file)[0])+ '.csv'), sep = ';')
else:
    print('No valid scenario given as input.')

##### Combine data into final output

In [ ]:
output_combined = pd.concat([output_landelijkeafvoer, output_overstorten], keys=['Defined Lateral Disc', 'Lateral disch.(m3/s)'], axis=1)
output_combined = output_combined[pd.to_datetime("2010-11-09 00:00:00", format='%Y-%m-%d %H:%M:%S'):pd.to_datetime("2010-11-21 23:00:00", format='%Y-%m-%d %H:%M:%S')]
output_final = output_combined.fillna(0)
output_final.to_csv(join(path_data, r'output_afvoeren\timeseries', 'combinedoutput.csv'), sep = ';')

In [ ]:
len(output_landelijkeafvoer.columns)
len(output_overstorten.columns)
len(output_final.columns)